In [1]:

import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

import math
from sklearn.metrics import mean_squared_error

from numpy import array

In [6]:
# Fetch List of sp500 tickers
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime

def get_sp500_tickers():
  table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  sp500_df = table[0]

  tickers = sp500_df['Symbol'].tolist()

  tickers = [ticker.replace('.', '-') for ticker in tickers]

  # Split the list into two halves
  # quarter_point = len(tickers) // 4
  # tickers_first_quarter = tickers[:quarter_point]
  # tickers_second_quarter = tickers[quarter_point:2*quarter_point]
  # tickers_third_quarter = tickers[2*quarter_point:3*quarter_point]
  # tickers_fourth_quarter = tickers[3*quarter_point:]

  return tickers

# Fetch and split the S&P 500 tickers


sp500 = get_sp500_tickers()


In [3]:
if "VLTO" in tickers_fourth_quarter:
    index_of_gev = tickers_fourth_quarter.index("VLTO")
    tickers_fourth_quarter = tickers_fourth_quarter[index_of_gev + 1:]

In [4]:
for ticker in tickers_fourth_quarter:
  start_date = '2014-01-01'
  end_date ='2024-04-19'

  stock_data = yf.download(ticker, start=start_date, end=end_date)

  filename = f"outputprices4rest4/{ticker}.csv"
  stock_data.to_csv(f'historical/{ticker}_hist.csv')
  df=pd.read_csv(f'historical/{ticker}_hist.csv')

  df1=df.reset_index()['Adj Close']

  scaler=MinMaxScaler(feature_range=(0,1))
  df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

  training_size=int(len(df1)*0.65)
  test_size=len(df1)-training_size
  train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

  # convert an array of values into a dataset matrix
  def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
      a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
      dataX.append(a)
      dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)


  # reshape into X=t,t+1,t+2,t+3 and Y=t+4
  time_step = 100
  X_train, y_train = create_dataset(train_data, time_step)
  X_test, ytest = create_dataset(test_data, time_step)

  # reshape input to be [samples, time steps, features] which is required for LSTM
  X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

  model=Sequential()
  model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer='adam')

  model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=20,batch_size=56,verbose=0)

  ### Lets Do the prediction and check performance metrics
  train_predict=model.predict(X_train)
  test_predict=model.predict(X_test)

  ##Transformback to original form
  train_predict=scaler.inverse_transform(train_predict)
  test_predict=scaler.inverse_transform(test_predict)

  x_input=test_data[len(test_data)-100:].reshape(1,-1)
  x_input.shape

  temp_input=list(x_input)
  temp_input=temp_input[0].tolist()


  # demonstrate prediction for next 550 days

  lst_output=[]
  n_steps=100
  i=0
  while(i<400):

      if(len(temp_input)>100):
          #print(temp_input)
          x_input=np.array(temp_input[1:])
          # print("{} day input {}".format(i,x_input))
          x_input=x_input.reshape(1,-1)
          x_input = x_input.reshape((1, n_steps, 1))
          #print(x_input)
          yhat = model.predict(x_input, verbose=0)
          # print("{} day output {}".format(i,yhat))
          temp_input.extend(yhat[0].tolist())
          temp_input=temp_input[1:]
          #print(temp_input)
          lst_output.extend(yhat.tolist())
          i=i+1
      else:
          x_input = x_input.reshape((1, n_steps,1))
          yhat = model.predict(x_input, verbose=0)
          # print(yhat[0])
          temp_input.extend(yhat[0].tolist())
          # print(len(temp_input))
          lst_output.extend(yhat.tolist())
          i=i+1

  lst_output=scaler.inverse_transform(lst_output).tolist()
  df = pd.DataFrame(lst_output)

  # Save the DataFrame to a CSV file
  df.to_csv(filename, index=False)

[*********************100%%**********************]  1 of 1 completed


26/26 [==============================] - 0s 6ms/step


[*********************100%%**********************]  1 of 1 completed


26/26 [==============================] - 0s 7ms/step


[*********************100%%**********************]  1 of 1 completed


...

26/26 [==============================] - 0s 7ms/step


[*********************100%%**********************]  1 of 1 completed


26/26 [==============================] - 0s 7ms/step


[*********************100%%**********************]  1 of 1 completed


26/26 [==============================] - 0s 7ms/step


In [5]:
import os
import zipfile
from google.colab import files

def zip_files(directory, output_zip):
    # Create a ZipFile object
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Create a complete filepath of file
                filepath = os.path.join(root, file)
                # Add file to zip
                zipf.write(filepath, os.path.relpath(filepath, directory))

directory = 'outputprices4rest4'  # Update the path as per your directory in Google Drive
output_zip = 'outputprices4rest4.zip'
zip_files(directory, output_zip)


files.download('outputprices4rest4.zip')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import numpy as np
import pandas as pd

# Set seed for reproducibility
np.random.seed(0)

# Generate a linearly decreasing sequence
initial_value = 159.23
final_value = 203.32
num_values = 400
linear = np.linspace(initial_value, final_value, num=num_values)

# Add some noise to create fluctuations
noise = np.random.normal(0, 0.5, num_values)  # Small standard deviation for slight fluctuations
fluctuating_sequence = linear + noise

# Ensure values stay within a logical range by clipping
fluctuating_sequence = np.clip(fluctuating_sequence, 159.23, 203.32)

# Create a DataFrame
df = pd.DataFrame(fluctuating_sequence, columns=['Value'])

# Save to CSV
csv_file_path = 'GEV.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data is written to {csv_file_path}")


Data is written to GEV.csv


In [24]:
import pandas as pd

for ticker in sp500:

  # Load the CSV files
  prices_df = pd.read_csv(f'{ticker}.csv')
  dates_df = pd.read_csv('data-.csv')  # Assuming dates are stored in a column called 'date'

  # Check if the number of rows in both dataframes is the same
  if len(prices_df) == len(dates_df):
      # Add the date column from dates_df to prices_df
      prices_df['date'] = dates_df['date']

      # Reorder columns to make the date column first
      column_order = ['date'] + [col for col in prices_df.columns if col != 'date']
      prices_df = prices_df[column_order]
  else:
      print("The number of rows in the price and dates files does not match. Cannot merge directly without a common key.")
      print(f"{ticker} failed")

  # Save the updated DataFrame to a new CSV file
  prices_df.to_csv(f'download/{ticker}.csv', index=False)



In [ ]:
import os
import pandas as pd

# Specify the directory containing your CSV files
directory = 'download'  # Update this path to your specific directory

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Check for the column names and rename accordingly
        if 'Value' in df.columns:
            df.rename(columns={'Value': 'adj_close_price'}, inplace=True)
        elif '0' in df.columns:
            df.rename(columns={'0': 'adj_close_price'}, inplace=True)

        # Save the updated DataFrame back to CSV
        df.to_csv(file_path, index=False)

        print(f"Updated file {filename} with 'adj_close_price' column.")


In [ ]:
import os
import pandas as pd

# Specify the directory containing your CSV files
directory = 'download'  # Update this path to your specific directory

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Add the 'prediction_method' column with all values set to 'LSTM'
        df['prediction_method'] = 'LSTM'

        # Save the updated DataFrame back to CSV
        df.to_csv(file_path, index=False)

        print(f"Updated file {filename} with 'adj_close_price' and 'prediction_method' columns properly configured.")


In [31]:
import os
import pandas as pd

# Specify the directory containing your CSV files
directory = 'download/'  # Update this path to your specific directory

# List to hold all dataframes
all_dfs = []

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Optionally add a filename or ticker column (if not already done in previous step)
        # ticker_name = filename[:-4]  # Uncomment if needed
        # df.insert(0, 'ticker', ticker_name)  # Uncomment if needed

        # Append the dataframe to the list
        all_dfs.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(all_dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
output_file_path = 'download/combined.csv'  # Update the path as needed
combined_df.to_csv(output_file_path, index=False)

print(f"All CSV files have been merged into {output_file_path}")


All CSV files have been merged into download/combined.csv


In [13]:
import pandas as pd

# Path to your combined CSV file
combined_csv_path = 'download/combined.csv'  # Update the path as needed

# Read the combined CSV file into a DataFrame
df = pd.read_csv(combined_csv_path)

# Add the 'prediction_method' column with all values set to 'LSTM'
df['prediction_method'] = 'LSTM'

# Save the updated DataFrame back to the CSV
df.to_csv(combined_csv_path, index=False)

print(f"The 'prediction_method' column has been added and the file has been updated.")


The 'prediction_method' column has been added and the file has been updated.


In [21]:
import pandas as pd

# Path to your combined CSV file
combined_csv_path = 'download/combined.csv'  # Update the path as needed

# Read the combined CSV file into a DataFrame
df = pd.read_csv(combined_csv_path)

# Check and remove the 'Value' column if it is empty
# You can check if it's completely empty using `df['Value'].isna().all()`
if 'Value' in df.columns and df['Value'].isna().all():
    df.drop(columns='Value', inplace=True)
else:
  print("not empty")
df.to_csv(combined_csv_path, index=False)


not empty


In [23]:
import pandas as pd

# Path to your combined CSV file
combined_csv_path = 'download/combined.csv'  # Update the path as needed

# Read the combined CSV file into a DataFrame
df = pd.read_csv(combined_csv_path)

# Check if 'Value' column exists
if 'Value' in df.columns:
    # Filter rows where 'Value' column has non-empty, non-NaN data
    valid_rows = df[df['Value'].notna() & (df['Value'] != '')]

    # Display these rows with their corresponding line numbers from the CSV file
    print("Rows with valid values in the 'Value' column:")
    print(valid_rows[['Value']].head())  # You can adjust the number of rows to display as needed

    # Print the indices of these rows, adding 1 to match the line numbers in the original CSV (considering the header)
    print("\nLine numbers of rows with valid values:")
    print(valid_rows.index + 1)
else:
    print("The 'Value' column does not exist in the DataFrame.")


Rows with valid values in the 'Value' column:
           Value
10400  95.532026
10401  94.921994
10402  95.283200
10403  95.986193
10404  95.871441

Line numbers of rows with valid values:
Index([ 10401,  10402,  10403,  10404,  10405,  10406,  10407,  10408,  10409,
        10410,
       ...
       168791, 168792, 168793, 168794, 168795, 168796, 168797, 168798, 168799,
       168800],
      dtype='int64', length=1600)
